In [1]:
from pytorch_lightning.trainer import Trainer
from model_interface import MInterface
from data_interface import DInterface

import pickle

/root/anaconda3/envs/dyMEAN/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 定义参数
args = {
    'res_dir': './results',
    'ex_name': 'debug',
    'check_val_every_n_epoch': 1,
    'dataset': 'PTM',
    'model_name': 'MeToken', # model name here
    'lr': 1e-4,
    'lr_scheduler': 'onecycle',
    'offline': 1,
    'seed': 114514,
    'batch_size': 16,
    'num_workers': 16,
    'pad': 1024,
    'min_length': 40,
    'path': './data_inference', # data path here
    'with_null_ptm': 1,
    'epoch': 20,
    'augment_eps': 0.0,
    'module_type': 94,
    'weight_type': 0,
    'gamma': 2.0,
    'final_tau': 1e-4,
    'pretrain': 0,
    'test_only': 1,
    'inference_pos':[[31]],
    'ckpt_from_deepspeed': 0,
    'ckpt_path': "pretrained_model/checkpoint.ckpt",
    'gpus': [0],
    'strategy': 'auto',
    'wandb_offline': 1
}

In [3]:
import json
from re import I
from Bio import PDB
import os
import h5py
import numpy as np

def extract_pdb_info_from_folder(folder_path, output_json_path):
    parser = PDB.PDBParser(QUIET=True)
    protein_list = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.pdb'):
            pdb_file_path = os.path.join(folder_path, filename)
            structure = parser.get_structure(filename.replace('.pdb', ''), pdb_file_path)
            
            seq = ''
            coords_chain_A = {'N_chain_A': [], 'C_chain_A': [], 'CA_chain_A': [], 'O_chain_A': []}
            
            for model in structure:
                for chain in model:
                    if chain.id == 'A':  # Process only chain A
                        for residue in chain:
                            if PDB.is_aa(residue):
                                # Get the sequence
                                seq += PDB.Polypeptide.three_to_one(residue.resname)
                                # Get the coordinates of N, C, CA, and O atoms
                                for atom in residue:
                                    if atom.id == 'N':
                                        coords_chain_A['N_chain_A'].append(atom.coord.tolist())
                                    elif atom.id == 'C':
                                        coords_chain_A['C_chain_A'].append(atom.coord.tolist())
                                    elif atom.id == 'CA':
                                        coords_chain_A['CA_chain_A'].append(atom.coord.tolist())
                                    elif atom.id == 'O':
                                        coords_chain_A['O_chain_A'].append(atom.coord.tolist())
            
            # Extract ID from the file name
            pdb_id = filename.replace('.pdb', '')
            
            # Create the JSON object for this PDB file
            protein_data = {
                "id": pdb_id,
                "seq": seq,
                "coords_chain_A": coords_chain_A
            }
            
            protein_list.append(protein_data)
    
    # Write the JSON list to file
    with open(output_json_path, 'w') as json_file:
        json.dump(protein_list, json_file, indent=4)

def process_coords_chain_A(group, coords_chain_A):
    for key, value in coords_chain_A.items():
        array = np.array(value, dtype=np.float32)
        group.create_dataset(key, data=array)

def apply_ptm_indices(input_json_path, output_json_path, ptm_indices):
    with open(input_json_path, 'r') as json_file:
        protein_list = json.load(json_file)

    for i,protein_data in enumerate(protein_list):
        seq_length = len(protein_data["seq"])
        
        if -1 in ptm_indices[i]:
            ptm = [1] * seq_length
        else:
            ptm = [0] * seq_length
            for index in ptm_indices[i]:
                if 0 <= index < seq_length:
                    ptm[index] = 1
        
        protein_data["ptm"] = ptm

    with open(output_json_path, 'w') as json_file:
        json.dump(protein_list, json_file, indent=4)

def dict_to_hdf5(group, item):
    for key, value in item.items():
        if key == 'coords_chain_A':
            coords_group = group.create_group(key)
            process_coords_chain_A(coords_group, value)
        elif isinstance(value, list):
            value = np.array(value)
            group.create_dataset(key, data=value)
        elif isinstance(value, (int, float)):
            value = np.array([value])
            group.create_dataset(key, data=value)
        elif isinstance(value, str):
            dt = h5py.special_dtype(vlen=str)
            value = np.array([value], dtype=dt)
            group.create_dataset(key, data=value)
        else:
            raise ValueError(f"Data type not supported: {type(value)}")

def json_to_hdf5(json_filepath, hdf5_filepath):
    with open(json_filepath, 'r') as json_file:
        data = json.load(json_file)
    
    with h5py.File(hdf5_filepath, 'w') as hdf5_file:
        for i, item in enumerate(data):
            group = hdf5_file.create_group(str(i))
            dict_to_hdf5(group, item)
            
# Example usage
pdb_file_path = args["path"]  # Replace with your PDB file path
output_json_path = args["path"]+'/predict.json'
output_path=args["path"]+"/predict.hdf5"
predict_indices=args["inference_pos"]
extract_pdb_info_from_folder(pdb_file_path, output_json_path)
apply_ptm_indices(output_json_path,output_json_path,predict_indices)
json_to_hdf5(output_json_path, output_path)
print("Data Prepared")

Data Prepared


In [4]:
data_module = DInterface(**args)
data_module.setup(stage="predict")


In [5]:
trainer_config = {
    'gpus': args['gpus'] if args['ex_name'] != 'debug' else [0],
    'max_epochs': args['epoch'],
    'strategy': args['strategy'],
    'accelerator': 'gpu',
    'resume_from_checkpoint': args['ckpt_path']
}

trainer = Trainer(**trainer_config)

model = MInterface.load_from_checkpoint(trainer_config["resume_from_checkpoint"], strict=False,**(args))
model.hparams["predict_indices"] = args["inference_pos"] # the line that comes with problem
result = trainer.predict(model, data_module)[0]
print(result)
with open("results/prediction_sample.pkl","wb") as f:
    pickle.dump(result,f)

/root/anaconda3/envs/dyMEAN/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  rank_zero_deprecation(
/root/anaconda3/envs/dyMEAN/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:55: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision 

Predicting DataLoader 0: 100%|██████████| 1/1 [00:01<00:00,  1.78s/it]
[{31: 16}]
